# Live cu-inj-live-impact 

In [ ]:
# Setup directories, and convert dashboard notebook to a script for importing
#!./setup.bash

In [ ]:
from impact import evaluate_impact_with_distgen, run_impact_with_distgen
from impact.tools import isotime
from impact.evaluate import  default_impact_merit
from impact import Impact

from make_dashboard import make_dashboard
from get_vcc_image import get_live_distgen_xy_dist

import matplotlib as mpl

In [ ]:
import pandas as pd

import json
import epics

import sys
import os
from time import sleep, time


import matplotlib.pyplot as plt

import matplotlib as mpl
mpl.use('Agg')

# Nicer plotting
# %config InlineBackend.figure_format = 'retina'

# Logging

In [ ]:
PREFIX = 'lume-impact-live-demo'

In [ ]:
import logging

# Gets or creates a logger
logger = logging.getLogger(PREFIX)  

# set log level
logger.setLevel(logging.INFO)

# define file handler and set formatter
file_handler = logging.FileHandler(f'{PREFIX}.log')
#formatter    = logging.Formatter('%(asctime)s : %(levelname)s : %(name)s : %(message)s')
formatter    = logging.Formatter(fmt="%(asctime)s :  %(name)s : %(message)s ", datefmt="%Y-%m-%dT%H:%M:%S%z")

# Add print to stdout
logger.addHandler(logging.StreamHandler(sys.stdout))

file_handler.setFormatter(formatter)

# add file handler to logger
logger.addHandler(file_handler)

# Configuration

Set up basic input sources and output path.

Depends on the `$LCLS_LATTICE` environmental variable pointing to a checked out copy of https://github.com/slaclab/lcls-lattice (protected)

In [ ]:
ROOT = os.getcwd()

In [ ]:
# Directory for summary output
OUTPUTDIR = os.path.join(ROOT, 'output')

# Directory to output plots
PLOTDIR = os.path.join(ROOT, 'plot')

# Directory for archive files
APATH = os.path.join(ROOT, 'archive')

# PV -> Sim conversion table
CSV =  'pv_mapping/cu_inj_impact.csv'

# Base input files
CONFIG0 = {'impact_config': os.path.expandvars('$LCLS_LATTICE/impact/models/cu_inj/ImpactT.yaml'),
          'distgen_input_file': os.path.expandvars('$LCLS_LATTICE/distgen/models/cu_inj/vcc_image/distgen.yaml')
         }

# Dummy file for distgen
DISTGEN_LASER_FILE = os.path.join(ROOT, 'distgen_laser.txt')

# Base settings
SETTINGS0 = {
 'distgen:t_dist:length:value': 4 * 1.65 ,     #  Inferred pulse stacker FWHM: 4 ps, converted to tukey length
 'distgen:n_particle': 100000,   
 'stop': 16.5, 
 'timeout': 100000,
 'header:Nx': 32,
 'header:Ny': 32,
 'header:Nz': 32,
 #'total_charge':0, # Set to 0 to turn off Space Charge for faster execution 
 'change_timestep_1:dt':4e-12, 
 
 # Parallel control
 'header:Nprow':4,
 'header:Npcol':8,
 'use_mpi':True,
 'mpi_run': 'mpirun -n {n} --use-hwthread-cpus {command_mpi}'
   }

# EPICS -> Simulation settings

In [ ]:
IRIS_DIAMETER_FROM_ENUM_IN_MM = {0:'LCLS', # LCLS cutout picture
       1 :1.8 ,
       2 :1.6 ,
       3 :1.4 ,
       4 :1.2 ,
       5 :1.1 ,
       6 :1.0 ,
       7 :0.8 ,
       8 :0.6 ,
       9 :0.5 ,
       10: 0.4 , 
       11: 0.3 }

def get_live_settings(csv, base_settings={}):
    """
    Fetches live settings for all devices in the CSV table, and translates them to simulation inputs
     
    """
    df = pd.read_csv(csv)
    pv_names = list(df['device_pv_name'])
    
    df['pv_value'] = epics.caget_many(pv_names)
    df.set_index('device_pv_name', inplace=True)
    
    # Custom for enum
    iris_enum = int(df.at['IRIS:LR20:130:CONFG_SEL', 'pv_value'])
    df.at['IRIS:LR20:130:CONFG_SEL', 'pv_value'] = IRIS_DIAMETER_FROM_ENUM_IN_MM[iris_enum]
    
    # Assign impact
    df['impact_value'] = df['impact_factor']*df['pv_value'] 

    # Collect settings
    settings = base_settings.copy()
    settings.update(dict(zip(df['impact_name'], df['impact_value'])))

    # VCC image
    settings['distgen:xy_dist:file'] = get_live_distgen_xy_dist(filename=DISTGEN_LASER_FILE)
    # We won't need this anymore
    settings.pop('distgen:r_dist:sigma_xy:value')
    
    
    return settings, df

LIVE_SETTINGS, DF = get_live_settings(CSV, SETTINGS0)
DF[['Variable', 'bmad_name', 'pv_value','pv_unit',  'device_min', 'device_max',  'impact_name', 'impact_factor', 'impact_unit',
       'impact_description',  'impact_value']]

In [ ]:
#%%time
#I0 = run_impact_with_distgen(LIVE_SETTINGS, **CONFIG0, verbose=True )
    

In [ ]:
#I0.plot(['sigma_x', 'sigma_y'], y2=['sigma_x'], ylim2=(0,2e-3), figsize=(16,9))

In [ ]:
#I0.initial_particles.plot('x', 'y')

# Get live values, run Impact-T, make dashboard

In [ ]:
# Patch this into the function below for the dashboard creation
def my_merit(impact_object, itime):
    # Collect standard output statistics
    merit0 = default_impact_merit(impact_object)
    # Make the dashboard from the evaluated object
    plot_file = make_dashboard(impact_object, itime=itime, outpath=PLOTDIR)
    #print('Dashboard written:', plot_file)
    logger.info(f'Dashboard written: {plot_file}')
    
    # Assign extra info
    merit0['plot_file'] = plot_file    
    merit0['isotime'] = itime
    
    # Clear any buffers
    plt.close('all')

    return merit0

In [ ]:
def run1():
    dat = {}
    
    # Acquire settings
    mysettings, df = get_live_settings(CSV, SETTINGS0)
    itime = isotime()
    dat['isotime'] = itime
    
    # Record inputs
    dat['inputs'] = mysettings
    dat['config'] = CONFIG0
    dat['pv_mapping_dataframe'] = df.to_dict()
    
    
    logger.info(f'Acquired settings from EPICS at: {itime}')
    logger.info(f'Running evaluate_impact_with_distgen...')
    
    t0 = time()
    dat['outputs'] = evaluate_impact_with_distgen(mysettings,
                                       merit_f=lambda x: my_merit(x, itime),
                                       archive_path=APATH,
                                       **CONFIG0 )
    logger.info(f'...finished in {(time()-t0)/60:.1f} min')
    fname = fname=f'{OUTPUTDIR}/{PREFIX}-{itime}.json'
    json.dump(dat, open(fname, 'w'))
    #print('Written:', fname)
    logger.info(f'Output written: {fname}')
    return dat
    

In [ ]:
# Here are the results
#result = run1()
#result.keys()

In [ ]:
# Basic config
#result['config']

In [ ]:
# Simulation inputs
#result['inputs']

In [ ]:
# Simulation outputs
#result['outputs']

# Show the plot 

In [ ]:
#from IPython.display import Image
#Image(filename=result['outputs']['plot_file']) 

# loop it


In [ ]:
while True:
    run1()